In [1]:
# To obtain the data, go to "http://pic2recipe.csail.mit.edu/"
# Follow the instructions to download the dataset, which will involve creating an account
# After that, go to "http://im2recipe.csail.mit.edu/dataset/download/"
# and download the link labeled "Layers"
# Place this file in the working directory and make sure it is named 
# "recipe1M_layers.tar.gz" (this is the default name).

In [2]:
import tarfile
import sqlite3
import json
import pandas as pd

In [3]:
# open tarfile and explore what is there
tar = tarfile.open("recipe1M_layers.tar.gz")
files = tar.getmembers()

In [4]:
files

[<TarInfo 'layer1.json' at 0x1e25a126ba8>,
 <TarInfo 'layer2.json' at 0x1e25a29f2b8>]

In [5]:
# extract recipe data into a string
f = tar.extractfile(files[0]).read()

In [6]:
# convert to a python list
temp = json.loads(f)

In [7]:
type(temp)

list

In [8]:
# what information do we have
temp[0].keys()

dict_keys(['ingredients', 'url', 'partition', 'title', 'id', 'instructions'])

In [27]:
# create the DataFrame
df = pd.DataFrame()

for key in temp[0].keys():
    tempList = [temp[i][key] for i in range(0, len(temp)-1)]
    df[key] = tempList

In [28]:
df.head(5)

,ingredients,url,partition,title,id,instructions
0,"[{'text': '6 ounces penne'}, {'text': '2 cups ...",http://www.epicurious.com/recipes/food/views/-...,train,Worlds Best Mac and Cheese,000018c8a5,[{'text': 'Preheat the oven to 350 F. Butter o...
1,"[{'text': '1 c. elbow macaroni'}, {'text': '1 ...",http://cookeatshare.com/recipes/dilly-macaroni...,train,Dilly Macaroni Salad Recipe,000033e39b,[{'text': 'Cook macaroni according to package ...
2,"[{'text': '8 tomatoes, quartered'}, {'text': '...",http://www.foodnetwork.com/recipes/gazpacho1.html,train,Gazpacho,000035f7ed,[{'text': 'Add the tomatoes to a food processo...
3,"[{'text': '2 12 cups milk'}, {'text': '1 12 cu...",http://www.food.com/recipe/crunchy-onion-potat...,test,Crunchy Onion Potato Bake,00003a70b1,[{'text': 'Preheat oven to 350 degrees Fahrenh...
4,[{'text': '1 (3 ounce) package watermelon gela...,http://www.food.com/recipe/cool-n-easy-creamy-...,train,Cool 'n Easy Creamy Watermelon Pie,00004320bb,"[{'text': 'Dissolve Jello in boiling water.'},..."


In [29]:
# unpack the ingredients and instructions columns
ingr_unpacked = []
istr_unpacked = []
for i in range(0, df.shape[0]): # loop over each row
    ingr_list = []
    istr_list = []
    for ingr_dict in df['ingredients'][i]: # loop over each ingredient
        ingr_list.append(ingr_dict['text']) # add to new list
    for istr_dict in df['instructions'][i]: # repeat for instructions
        istr_list.append(istr_dict['text'])
    ingr_str = json.dumps(ingr_list) # convert to JSON string format
    istr_str = json.dumps(istr_list)
    ingr_unpacked.append(ingr_str) # add the string just constructed to another list
    istr_unpacked.append(istr_str)

df["ingredients"] = ingr_unpacked # replace columns
df["instructions"] = istr_unpacked

In [30]:
# check data
df.head(5)

,ingredients,url,partition,title,id,instructions
0,"[""6 ounces penne"", ""2 cups Beechers Flagship C...",http://www.epicurious.com/recipes/food/views/-...,train,Worlds Best Mac and Cheese,000018c8a5,"[""Preheat the oven to 350 F. Butter or oil an ..."
1,"[""1 c. elbow macaroni"", ""1 c. cubed American c...",http://cookeatshare.com/recipes/dilly-macaroni...,train,Dilly Macaroni Salad Recipe,000033e39b,"[""Cook macaroni according to package direction..."
2,"[""8 tomatoes, quartered"", ""Kosher salt"", ""1 re...",http://www.foodnetwork.com/recipes/gazpacho1.html,train,Gazpacho,000035f7ed,"[""Add the tomatoes to a food processor with a ..."
3,"[""2 12 cups milk"", ""1 12 cups water"", ""14 cup ...",http://www.food.com/recipe/crunchy-onion-potat...,test,Crunchy Onion Potato Bake,00003a70b1,"[""Preheat oven to 350 degrees Fahrenheit."", ""S..."
4,"[""1 (3 ounce) package watermelon gelatin"", ""14...",http://www.food.com/recipe/cool-n-easy-creamy-...,train,Cool 'n Easy Creamy Watermelon Pie,00004320bb,"[""Dissolve Jello in boiling water."", ""Allow to..."


In [32]:
type(df['ingredients'][0])

str

In [33]:
conn = sqlite3.connect("recipes1M.db")

In [34]:
df.to_sql("recipes", conn, if_exists = "replace", index = False, chunksize = 20000)

In [35]:
# verify it worked
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('recipes',)]


In [36]:
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")

for result in cursor.fetchall():
    print(result[0])

CREATE TABLE "recipes" (
"ingredients" TEXT,
  "url" TEXT,
  "partition" TEXT,
  "title" TEXT,
  "id" TEXT,
  "instructions" TEXT
)


In [37]:
conn.close()

In [75]:
# preliminary query function
def title_keyword_search(keyword):
    ''' Searches the recipe databse for entries with the keyword in the title. '''
    
    conn = sqlite3.connect("recipes1M.db")
    cmd = \
    f"""
    SELECT R.title, R.url
    FROM recipes R
    WHERE R.title LIKE "%{keyword}%"
    """
    
    df = pd.read_sql_query(cmd, conn)
    conn.close()
    return df

In [79]:
title_keyword_search("steak")

,title,url
0,Steak & Asparagus Wraps,http://www.kraftrecipes.com/recipes/steak-aspa...
1,Erin's Mashed Potatoes,http://www.food.com/recipe/erins-mashed-potato...
2,Caramelized Beef Skewers,http://www.kraftrecipes.com/recipes/caramelize...
3,BBq Steak Sandwiches With a Rainbow of Peppers,http://www.food.com/recipe/bbq-steak-sandwiche...
4,Lemon Butter for Steak,http://www.epicurious.com/recipes/food/views/l...
...,...,...
17434,Simply Scrumptious Stilton Steak,http://www.food.com/recipe/simply-scrumptious-...
17435,Garlic Mexican Rub,http://www.food.com/recipe/garlic-mexican-rub-...
17436,Monkey Meat,http://www.food.com/recipe/monkey-meat-176933
17437,Latin-Spiced Rib Eye with Sauteed Onions and C...,http://www.foodandwine.com/recipes/latin-spice...
